In [28]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from PIL import Image
from six.moves import cPickle as pickle
from six.moves import range

In [29]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [27]:
# visiualization
a = np.uint8(train_dataset[2] * 255 + 128)
a = Image.fromarray(a)
a.show()

In [30]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  #One Hot encoding
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [31]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [36]:
#CNN with strides = 2
batch_size = 16
patch_size = 5
depth1 = 32
num_hidden1 = 64

graph = tf.Graph()

def conv2d(x, w):
  return tf.nn.conv2d(x, w, [1,1,1,1], padding='SAME')

def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
#   layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
#   layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 2 * image_size // 2 * depth1, num_hidden1], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
#   layer4_weights = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev=0.1))
#   layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden1, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob = 0.5):
    h = max_pool(tf.nn.relu(conv2d(data, layer1_weights) + layer1_biases))
#     h = max_pool(tf.nn.relu(conv2d(h, layer2_weights) + layer2_biases))
    h = tf.reshape(h, [-1, image_size / 2 * image_size / 2 * depth1])
    h = tf.nn.relu(tf.matmul(h, layer3_weights) + layer3_biases)
    h = tf.nn.dropout(h, keep_prob)
#     h = tf.nn.relu(tf.matmul(h, layer4_weights) + layer4_biases)
#     h = tf.nn.dropout(h, keep_prob)
    return tf.matmul(h, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [37]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.462592
Minibatch accuracy: 12.5%
Validation accuracy: 12.6%
Minibatch loss at step 50: 2.209792
Minibatch accuracy: 18.8%
Validation accuracy: 37.5%
Minibatch loss at step 100: 2.172244
Minibatch accuracy: 25.0%
Validation accuracy: 54.6%
Minibatch loss at step 150: 1.646758
Minibatch accuracy: 43.8%
Validation accuracy: 62.3%
Minibatch loss at step 200: 1.523140
Minibatch accuracy: 56.2%
Validation accuracy: 63.3%
Minibatch loss at step 250: 1.837822
Minibatch accuracy: 31.2%
Validation accuracy: 70.0%
Minibatch loss at step 300: 1.189128
Minibatch accuracy: 62.5%
Validation accuracy: 70.0%
Minibatch loss at step 350: 1.115103
Minibatch accuracy: 68.8%
Validation accuracy: 73.2%
Minibatch loss at step 400: 0.922086
Minibatch accuracy: 68.8%
Validation accuracy: 73.7%
Minibatch loss at step 450: 1.356073
Minibatch accuracy: 56.2%
Validation accuracy: 75.8%
Minibatch loss at step 500: 1.062090
Minibatch accuracy: 75.0%
Validation accuracy: 76.3%
M